# Libraries

In [19]:
import os
import random
import numpy as np
import pandas as pd
import torch
import transformers
import unicodedata
from datasets import load_dataset

# Config

In [20]:
seed = 42

hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_sun_70_15_15_stratify_v2'

aug_type = 'go_emotions'
aug_data_paths = [
    'data/go_emotions_sun/fear_single_242_comb_79_translated_cleaned_curated_merged_final.csv',
    'data/go_emotions_sun/anger_single_237_comb_73_translated_cleaned_curated_merged_final.csv',
    'data/go_emotions_sun/disgust_single_210_comb_35_translated_cleaned_curated_merged_final.csv',
    'data/go_emotions_sun/sadness_single_183_comb_20_translated_cleaned_curated_merged_final.csv',
    'data/go_emotions_sun/surprise_single_240_comb_22_translated_cleaned_curated_merged_final.csv',
    'data/go_emotions_sun/neutral_single_210_comb_0_translated_cleaned_curated_merged_final.csv',
]

In [21]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [22]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
splits = [*datasets.keys()]

print("Splits:", splits)
print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Splits: ['train', 'val', 'test']
Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [25]:
df = {split: pd.DataFrame(datasets[split]) for split in splits}
full_df_ = pd.concat(df.values())
print("Full DF size:", len(full_df_))
full_df_.head()

Full DF size: 924


,text,emotion,marah,jijik,takut,senang,sedih,terkejut,biasa
0,Aya randa ker nguseup Pantun sunda meuni reuseup,senang,0,0,0,1,0,0,0
1,pastina ath mang ku abdi shere ken knu grup + SW,biasa,0,0,0,0,0,0,1
2,Mang Fiksi téh urang Majalengka ogé?,senang,0,0,0,1,0,0,0
3,"mang dana ,uing can ngopi, kumaha ieu?",senang,0,0,0,1,0,0,0
4,Sedih Mang Ai kudu D Caritakeun Mah😄🙏,"senang, sedih",0,0,0,1,1,0,0


In [26]:
if 'aug_data_paths' in globals():
    aug_df = pd.concat([pd.read_csv(aug_data_path) for aug_data_path in globals()['aug_data_paths']])
    aug_df.drop(['id', 'curation_status', 'text', 'num_emotions'], axis=1, inplace=True)
    aug_df.rename(columns={'text_translated': 'text'}, inplace=True)
    aug_df['emotion'] = aug_df.apply(lambda row: ', '.join([col for col in emotion_cols if row[col] == 1]), axis=1)
    aug_df[f'aug_{aug_type}'] = 1
    print("Augmentation DF size:", len(aug_df))
    
    full_df_[f'aug_{aug_type}'] = 0
    full_df = pd.concat([full_df_, aug_df])
    print("Full DF size (after augmentation):", len(full_df))
else:
    full_df = full_df_

full_df.head()

Augmentation DF size: 1571
Full DF size (after augmentation): 2495


,text,emotion,marah,jijik,takut,senang,sedih,terkejut,biasa,aug_go_emotions
0,Aya randa ker nguseup Pantun sunda meuni reuseup,senang,0,0,0,1,0,0,0,0
1,pastina ath mang ku abdi shere ken knu grup + SW,biasa,0,0,0,0,0,0,1,0
2,Mang Fiksi téh urang Majalengka ogé?,senang,0,0,0,1,0,0,0,0
3,"mang dana ,uing can ngopi, kumaha ieu?",senang,0,0,0,1,0,0,0,0
4,Sedih Mang Ai kudu D Caritakeun Mah😄🙏,"senang, sedih",0,0,0,1,1,0,0,0


## EDA

In [27]:
def contains_non_ascii(text):
    try:
        text.encode('ascii')
    except UnicodeEncodeError:
        return True
    return False

print("Total data with non-ASCII chars:", int(full_df['text'].apply(contains_non_ascii).sum()))

Total data with non-ASCII chars: 1154


In [31]:
def normalize_to_ascii(text):
    return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')

# Normalize to ASCII equivalents
full_df['text'] = full_df['text'].apply(normalize_to_ascii)
print("Total data with non-ASCII chars (after normalizing them):", int(full_df['text'].apply(contains_non_ascii).sum()))

Total data with non-ASCII chars (after normalizing them): 0


In [32]:
print("Distribution of emotions:")
full_df[emotion_cols].sum().sort_values(ascending=False)

Distribution of emotions:


senang      677
terkejut    509
sedih       482
marah       459
jijik       418
takut       378
biasa       253
dtype: int64

In [33]:
print("Emotion combinations distribution:")
full_df['emotion'].value_counts()

Emotion combinations distribution:


emotion
senang                                  434
sedih                                   266
terkejut                                256
biasa                                   253
takut                                   248
marah                                   246
jijik                                   221
senang, terkejut                        132
marah, jijik                             94
senang, sedih                            45
takut, sedih                             44
marah, sedih                             31
sedih, terkejut                          20
jijik, sedih                             20
jijik, takut                             20
takut, terkejut                          14
marah, takut                             14
marah, terkejut                          13
senang, sedih, terkejut                  11
jijik, terkejut                          10
jijik, senang                            10
marah, sedih, terkejut                    8
marah, jijik, terkejut  

In [34]:
full_df['num_emotions'] = full_df.apply(lambda row: int(sum(row[emotion_cols].tolist())), axis=1)

single_emotion_totals = [len(full_df[(full_df[emotion_col] == 1) & (full_df['num_emotions'] == 1)]) for emotion_col in emotion_cols]
comb_emotion_totals = [len(full_df[(full_df[emotion_col] == 1) & (full_df['num_emotions'] > 1)]) for emotion_col in emotion_cols]

min_single_emotion_total = float('inf')

print("Distribution of data with single emotion and combined emotions:\n")
for emotion, single_emotion_total, comb_emotion_total in zip(emotion_cols, single_emotion_totals, comb_emotion_totals):
    if single_emotion_total < min_single_emotion_total:
        min_single_emotion_total = single_emotion_total

    diff_from_max_single = max(comb_emotion_totals) - single_emotion_total
    diff_from_max_comb = max(comb_emotion_totals) - comb_emotion_total
    ratio = single_emotion_total / (comb_emotion_total if comb_emotion_total else 1) * 100
    total = single_emotion_total + comb_emotion_total

    print(f"{emotion:<15}:", single_emotion_total, "\t-> Difference from max:", diff_from_max_single, end="")
    print()

    print(f"{emotion + ' + ...':<15}:", comb_emotion_total, end="")
    if emotion != "biasa":
        print("\t-> Difference from max:", diff_from_max_comb, end="")
    print()

    print("-"*15)
    print("Total:", total)

    if emotion != "biasa":
        print(f"Ratio: {ratio:.3f}%")
    print()

print("Min. single emotion total:", min_single_emotion_total)

Distribution of data with single emotion and combined emotions:

marah          : 246 	-> Difference from max: 7
marah + ...    : 213	-> Difference from max: 40
---------------
Total: 459
Ratio: 115.493%

jijik          : 221 	-> Difference from max: 32
jijik + ...    : 197	-> Difference from max: 56
---------------
Total: 418
Ratio: 112.183%

takut          : 248 	-> Difference from max: 5
takut + ...    : 130	-> Difference from max: 123
---------------
Total: 378
Ratio: 190.769%

senang         : 434 	-> Difference from max: -181
senang + ...   : 243	-> Difference from max: 10
---------------
Total: 677
Ratio: 178.601%

sedih          : 266 	-> Difference from max: -13
sedih + ...    : 216	-> Difference from max: 37
---------------
Total: 482
Ratio: 123.148%

terkejut       : 256 	-> Difference from max: -3
terkejut + ... : 253	-> Difference from max: 0
---------------
Total: 509
Ratio: 101.186%

biasa          : 253 	-> Difference from max: 0
biasa + ...    : 0
---------------
Total

## Balance Data

In [35]:
# balanced_dfs = []

# for emotion in emotion_cols:
#     single_emotion_df = full_df[(full_df[emotion] == 1) & (full_df['num_emotions'] == 1)]
#     print(f"Single {emotion} DF:", len(single_emotion_df))
#     single_emotion_df_balanced = single_emotion_df.sample(n=min_single_emotion_total, random_state=seed)
#     print(f"Single {emotion} DF (after balancing):", len(single_emotion_df_balanced))

#     comb_emotion_df = full_df[(full_df[emotion] == 1) & (full_df['num_emotions'] > 1)]
#     print(f"Combination {emotion} DF:", len(comb_emotion_df))
#     comb_emotion_df_balanced = comb_emotion_df.sample(n=min_single_emotion_total, random_state=seed) if len(comb_emotion_df) > min_single_emotion_total else comb_emotion_df
#     print(f"Combination {emotion} DF (after balancing):", len(comb_emotion_df_balanced))

#     total = len(single_emotion_df_balanced) + len(comb_emotion_df_balanced)

#     print("-" * 46)
#     print("Total:", total)

#     balanced_dfs += [single_emotion_df_balanced, comb_emotion_df_balanced]

#     print()

# assert len(balanced_dfs) == len(emotion_cols) * 2

In [36]:
# balanced_df = pd.concat(balanced_dfs)
# balanced_df = balanced_df[~balanced_df.index.duplicated(keep='first')]

# print("DF size:", len(full_df))
# print("DF size (after balancing)", len(balanced_df))

In [37]:
# save_df = balanced_df.drop('num_emotions', axis=1)
# save_df.to_csv('sun_go_emotions_balanced.csv', index=False)

## Save Data

In [38]:
save_path = 'sun_go_emotions_v2.csv'
save_df = full_df.drop('num_emotions', axis=1)
save_df.to_csv(save_path, index=False)
print("Saved to:", save_path)

Saved to: sun_go_emotions_v2.csv
